## Course Project Jupyter Notebook

#### Data Files 
- business.csv 
- sample_submission.csv
- test_queries.csv
- train_reviews.csv
- user.csv
- validate_queries.csv

In [410]:
import pandas as pd
import numpy as np

## Preprocessing Business Data

Expects the csv file to be in an "all" folder in the working directory of this notebook

In [411]:
business_df = pd.read_csv("all/business.csv")
business_df_replace = business_df.copy()
# default value to replace for ambience when it is Nan
ambience_default = str({'romantic': False, 'intimate': False, 'classy': False, 'hipster': False, 'divey': False, 'touristy': False, 'trendy': False, 'upscale': False, 'casual': False})
business_df

,address,attributes,attributes_AcceptsInsurance,attributes_AgesAllowed,attributes_Alcohol,attributes_Ambience,attributes_BYOB,attributes_BYOBCorkage,attributes_BestNights,attributes_BikeParking,...,hours_Wednesday,is_open,latitude,longitude,name,neighborhood,postal_code,review_count,stars,state
0,595 Markham Street,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,17:0-1:0,0,43.664125,-79.411886,Southern Accent Restaurant,Palmerston,M6G 2L7,146,4.0,ON
1,2801 N 15th Ave,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",False,yes_free,"{'monday': False, 'tuesday': True, 'friday': T...",True,...,11:0-22:0,1,33.479807,-112.091188,Original Hamburger Works,NaN,85007,277,4.0,AZ
2,"5508 County Rd N, Ste 3",NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,False,...,11:0-0:0,1,43.149488,-89.206641,Chicken Lips,NaN,53590,102,4.5,WI
3,2227 N Rampart Blvd,NaN,NaN,NaN,beer_and_wine,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,7:0-19:0,1,36.201990,-115.283122,Omelet House Summerlin,Summerlin,89128,242,4.0,NV
4,1111 W Bell Rd,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,11:0-22:0,1,33.639774,-112.087738,Manuel's Mexican Restaurant & Cantina - Bell Rd,NaN,85023,230,3.5,AZ
5,1001 New Beginnings Dr,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,9:0-17:0,1,36.080453,-115.038166,Central Church - Henderson,NaN,89011,113,4.0,NV
6,5440 Walnut St,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,11:0-22:0,1,40.450866,-79.933919,China Palace,Shadyside,15232,110,3.0,PA
7,81 Underhill Drive,NaN,NaN,NaN,none,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,NaN,True,...,11:30-22:0,1,43.745928,-79.324623,Allwyn's Bakery,NaN,M3A 1K8,105,4.0,ON
8,2523 South Blvd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,20:0-6:0,1,35.202363,-80.864662,Long Animal Hospital,South End,28203,103,3.5,NC
9,6316 N Scottsdale Rd,NaN,NaN,NaN,full_bar,"{'romantic': False, 'intimate': False, 'classy...",NaN,NaN,"{'monday': False, 'tuesday': False, 'friday': ...",False,...,16:30-21:30,1,33.530358,-111.925905,Fat Ox,NaN,85253,234,4.0,AZ


### Feature Selection

I decided that we are only attempting to use features when more than half of its values are not Nan as otherwise, there are too less datapoints with a value. This percent non-Nan requirement can be changed nevertheless. 

I also decided to not look at the hours as it seems to complex to make into numerical value and would not help much in determining a user's review from intuition. The same is the case with the latitude, longitude, name, and address features. 

In [412]:
business_df_replace.drop([col for col in business_df.columns if business_df[col].isnull().sum() > 0.5 * 12058 ], axis=1, inplace=True)
business_df_replace.drop(['hours_Friday', 'hours_Monday', 'hours_Saturday',
       'hours_Sunday', 'hours_Thursday', 'hours_Tuesday', 'hours_Wednesday',
       'is_open', 'latitude', 'longitude', 'postal_code', 'name', 'address'], axis=1, inplace=True)
business_df_replace.columns

Index(['attributes_Alcohol', 'attributes_Ambience', 'attributes_BikeParking',
       'attributes_BusinessAcceptsCreditCards', 'attributes_BusinessParking',
       'attributes_Caters', 'attributes_GoodForKids', 'attributes_GoodForMeal',
       'attributes_HasTV', 'attributes_NoiseLevel',
       'attributes_OutdoorSeating', 'attributes_RestaurantsAttire',
       'attributes_RestaurantsDelivery', 'attributes_RestaurantsGoodForGroups',
       'attributes_RestaurantsPriceRange2',
       'attributes_RestaurantsReservations',
       'attributes_RestaurantsTableService', 'attributes_RestaurantsTakeOut',
       'attributes_WheelchairAccessible', 'attributes_WiFi', 'business_id',
       'categories', 'city', 'review_count', 'stars', 'state'],
      dtype='object')

###  Helper Functions

##### view_column_values
Helps to view what values occur inside the column of a dataframe

##### expand_dict_to_columns
Sometimes there are columns in the dataframe in which the data is a dictionary string(such as attributes_Ambience). This function helps expand that dictionary string into extra columns with the column being the key and the row content being the value. It returns the modified dataframe.

##### replace_column_nan
There are many Nan in the data. This function replaces the Nan of a specifc column of a dataframe with one of the values that already occur. The third parameter index_of_value_count is used to specify what value to replace, the values which can be viewed using view_column_values


In [413]:
def view_column_values(df, column_name):
    return df[column_name].value_counts()

def expand_dict_to_columns(df, column_name):
    expanded_df = df[column_name].apply(lambda x : dict(eval(x))).apply(pd.Series)
    expanded_df.fillna(False, inplace=True)
    df = pd.concat([df, expanded_df], axis = 1)
    df.drop([column_name], axis=1, inplace=True)
    return df

def replace_column_nan(df, column_name, index_of_value_count):
    df[column_name] = df[column_name].fillna(df[column_name].value_counts().index[index_of_value_count])

In [414]:
view_column_values(business_df, 'stars')

4.0    4258
3.5    3052
4.5    2228
3.0    1389
2.5     487
5.0     416
2.0     151
1.5      64
1.0      13
Name: stars, dtype: int64

In [415]:
business_df['stars'].isnull().sum()

0

#### Replacing all the NaN

In [416]:
business_df_replace['attributes_Ambience'] = business_df_replace['attributes_Ambience'].fillna(ambience_default)
business_df_replace =expand_dict_to_columns(business_df_replace, 'attributes_Ambience')
replace_column_nan(business_df_replace, 'attributes_Alcohol', 0)      # default full_bar, to change to none, change last parameter to 1 
replace_column_nan(business_df_replace, 'attributes_BikeParking', 0)  # default yes parking
replace_column_nan(business_df_replace, 'attributes_BusinessAcceptsCreditCards', 1)  # default True
replace_column_nan(business_df_replace, 'attributes_BusinessParking', 0)  # default just lot parking
business_df_replace = expand_dict_to_columns(business_df_replace, 'attributes_BusinessParking')
replace_column_nan(business_df_replace, 'attributes_Caters', 0)  # default True
replace_column_nan(business_df_replace, 'attributes_GoodForKids', 0)  # default True
replace_column_nan(business_df_replace, 'attributes_HasTV', 0)  # default True
replace_column_nan(business_df_replace, 'attributes_NoiseLevel', 0)  # default Average
replace_column_nan(business_df_replace, 'attributes_OutdoorSeating', 0)  # default True
replace_column_nan(business_df_replace, 'attributes_GoodForMeal', 0)  # default good for lunch and dinner
business_df_replace = expand_dict_to_columns(business_df_replace, 'attributes_GoodForMeal')
replace_column_nan(business_df_replace, 'attributes_RestaurantsAttire', 0)  # default casual
replace_column_nan(business_df_replace, 'attributes_RestaurantsDelivery', 0)  # default false
replace_column_nan(business_df_replace, 'attributes_RestaurantsGoodForGroups', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_RestaurantsPriceRange2', 0)  # default 2$ signs
replace_column_nan(business_df_replace, 'attributes_RestaurantsReservations', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_RestaurantsTableService', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_RestaurantsTakeOut', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_WheelchairAccessible', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_WiFi', 0)  # default free
replace_column_nan(business_df_replace, 'attributes_RestaurantsTableService', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_RestaurantsTableService', 0)  # default true
replace_column_nan(business_df_replace, 'attributes_RestaurantsTableService', 0)  # default true

#### Changing categorical input to numerical

In [420]:
for col in business_df_replace.columns:
    if col not in ['business_id', 'stars']: 
        if business_df_replace[col].dtypes == bool:
            # true becomes 1, false becomes 0
            business_df_replace[col] *=1
        elif business_df_replace[col].dtypes != np.dtype('int32') and business_df_replace[col].dtypes != np.dtype('int64') and business_df_replace[col].dtypes != float: 
            #changes categorical values to numerical values
            business_df_replace[col] = business_df_replace[col].astype('category').cat.codes

In [426]:
#note, the business_id column is not numerical, I kept it the same
business_df_replace.isnull().sum()

attributes_Alcohol                       0
attributes_BikeParking                   0
attributes_BusinessAcceptsCreditCards    0
attributes_Caters                        0
attributes_GoodForKids                   0
attributes_HasTV                         0
attributes_NoiseLevel                    0
attributes_OutdoorSeating                0
attributes_RestaurantsAttire             0
attributes_RestaurantsDelivery           0
attributes_RestaurantsGoodForGroups      0
attributes_RestaurantsPriceRange2        0
attributes_RestaurantsReservations       0
attributes_RestaurantsTableService       0
attributes_RestaurantsTakeOut            0
attributes_WheelchairAccessible          0
attributes_WiFi                          0
business_id                              0
categories                               0
city                                     0
review_count                             0
stars                                    0
state                                    0
romantic   

In [431]:
# USE THIS
business_df_replace

,attributes_Alcohol,attributes_BikeParking,attributes_BusinessAcceptsCreditCards,attributes_Caters,attributes_GoodForKids,attributes_HasTV,attributes_NoiseLevel,attributes_OutdoorSeating,attributes_RestaurantsAttire,attributes_RestaurantsDelivery,...,street,validated,lot,valet,dessert,latenight,lunch,dinner,breakfast,brunch
0,1,1,1,1,0,0,0,1,0,0,...,1,0,0,0,0,0,0,1,0,0
1,1,1,1,0,1,1,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
2,1,0,0,0,0,1,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
3,0,1,1,0,1,1,0,0,0,0,...,0,0,0,0,0,0,1,0,1,1
4,1,1,1,1,1,1,0,0,0,0,...,0,0,1,0,0,0,1,1,0,0
5,1,1,0,1,1,1,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
6,1,1,1,1,1,1,0,0,0,1,...,1,0,0,0,0,0,0,1,0,0
7,2,1,0,1,1,0,0,0,0,0,...,0,0,1,0,0,0,1,0,0,0
8,1,1,0,1,1,1,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
9,1,0,1,1,0,1,3,1,1,0,...,0,0,0,1,0,0,0,1,0,0


## Preprocessing User data
The users are mostly good for numerical features. However the following features 'elite', 'friends', 'name', 'yelping_since' are not but I decided to drop them as they do not intuitively seem super important. 

In [422]:
users_df = pd.read_csv("all/users.csv")
users_df_replace = users_df.copy()
users_df_replace.drop([ 'elite', 'friends', 'name', 'yelping_since'], axis=1, inplace=True)
users_df_replace.columns

Index(['average_stars', 'compliment_cool', 'compliment_cute',
       'compliment_funny', 'compliment_hot', 'compliment_list',
       'compliment_more', 'compliment_note', 'compliment_photos',
       'compliment_plain', 'compliment_profile', 'compliment_writer', 'cool',
       'fans', 'funny', 'review_count', 'useful', 'user_id'],
      dtype='object')

In [423]:
#check that all data is numerical, should output nothing if it is
for col in users_df_replace.columns:
    if col not in ['user_id']: 
        if users_df_replace[col].dtypes != np.dtype('int32') and users_df_replace[col].dtypes != np.dtype('int64') and users_df_replace[col].dtypes != float:
            print(col)

In [424]:
#check that there are no Nan values
users_df_replace.isnull().sum()

average_stars         0
compliment_cool       0
compliment_cute       0
compliment_funny      0
compliment_hot        0
compliment_list       0
compliment_more       0
compliment_note       0
compliment_photos     0
compliment_plain      0
compliment_profile    0
compliment_writer     0
cool                  0
fans                  0
funny                 0
review_count          0
useful                0
user_id               0
dtype: int64

In [430]:
# use this
users_df_replace

,average_stars,compliment_cool,compliment_cute,compliment_funny,compliment_hot,compliment_list,compliment_more,compliment_note,compliment_photos,compliment_plain,compliment_profile,compliment_writer,cool,fans,funny,review_count,useful,user_id
0,2.83,0,0,0,0,0,0,0,0,1,0,0,0,0,2,6,7,UxfpKHGO2dfQCdS9xLLJow
1,3.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,Kr5NDQFPPB_01-5CDmSqVg
2,3.09,0,0,0,0,0,0,0,0,0,0,0,0,1,0,10,2,wfoeMtriLwZsdRzcxNTaFA
3,4.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,aXb0kCIsIbPEEUSGomrrmA
4,4.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,sLrX2KGu3lc_JczAnsg0_Q
5,3.33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,12,3,nmYitfmo-pQ1hJWDnTLwGg
6,4.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,5tm0BfJEWGJWowr3sPGb8Q
7,1.50,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,0,PzHuq79aP6G25kEv-hejOA
8,1.00,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,GVYg18F-Rkuk63hvtHoG5Q
9,2.33,0,0,0,0,0,0,0,0,0,0,0,0,0,0,3,0,0IqKVB1rbaDyz0wlefmiAA


## Preprocessing Review Data

In [428]:
sample_submission = pd.read_csv("all/sample_submission.csv")
train_reviews = pd.read_csv("all/train_reviews.csv")

In [429]:
train_reviews_replace = train_reviews.copy()
train_reviews_replace.columns

Index(['business_id', 'cool', 'date', 'funny', 'review_id', 'stars', 'text',
       'useful', 'user_id'],
      dtype='object')

In [55]:
validate_df = pd.read_csv("all/validate_queries.csv")

In [56]:
validate_df

,Unnamed: 0,user_id,business_id,stars
0,146220,VHNFsnAvK9WGK1qkmm-9jQ,I4Nr-MVc26qWr08-S3Q1ow,2.0
1,193110,pvfD9IlV6PDJeA7dKdRDUA,ufA5ebLmVAqCuhMrApaONg,5.0
2,237052,IVo_5i_pBbQGnlmkvKRk-g,YkOCo5ipV2he2WXIAlZb-A,5.0
3,47545,LldI-DgOfmK9MDD6vuaW5Q,YOD9dXrnpu8HTRILpF0onw,5.0
4,80477,6DEqUrRhsfjX3rA0yu0akg,-5L8zOxibac-vBrsYtxXbQ,4.0
5,233884,LrbvGKEH0XMBxJHXFW5THg,X91rjC4EkH_3ZGY3ciIs8Q,5.0
6,157456,I4-FC6JpgI12BZlanvNv4g,VGRn5KgYKMkqdIKtBUzQQA,2.0
7,63547,J0wzD9Jp8_7rWa14opubMA,uUq0bMEjt202bMbRHkIcNA,5.0
8,119245,GL0o95jlTawH53za2DRKNQ,Ku_qQVoMA02kFZiu55_UwA,4.0
9,110040,ChaSfqm3RD6oSXKe42dtMQ,CYbrDESVuf8wpxRbxgBMJg,4.0
